THIS FILE CONTAINS THE CODE FOR MAKING THE TEST DATA READY FOR PREDICTION.IT INCLUDES THE FEATURE ENGINEERING STEPS  TIMESTAMP ALIGNMENT OF THE AIR-TEMPERATURE AND FILLING UP THE MISSING VALUES.

                        **Importing the necessary files using the kaggle API**

In [ ]:
from google.colab import files
file=files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp /content/kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c ashrae-energy-prediction

  0% 0.00/1.27M [00:00<?, ?B/s]
100% 1.27M/1.27M [00:00<00:00, 43.1MB/s]
100% 120M/120M [00:01<00:00, 97.1MB/s]

 95% 159M/167M [00:01<00:00, 104MB/s] 
100% 167M/167M [00:01<00:00, 101MB/s]
  0% 0.00/44.5k [00:00<?, ?B/s]
100% 44.5k/44.5k [00:00<00:00, 44.4MB/s]
  0% 0.00/2.53M [00:00<?, ?B/s]
100% 2.53M/2.53M [00:00<00:00, 167MB/s]
 94% 83.0M/88.4M [00:00<00:00, 103MB/s] 
100% 88.4M/88.4M [00:00<00:00, 121MB/s]


In [ ]:
!unzip /content/test.csv.zip

Archive:  /content/test.csv.zip
  inflating: test.csv                


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle

In [ ]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import lightgbm as lgb

**Function for reducing the memory usage**

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
df_test=pd.read_csv('test.csv')

In [ ]:
df_test_red=reduce_mem_usage(df_test,verbose=True)

Mem. usage decreased to 596.49 Mb (53.1% reduction)


In [ ]:
del df_test

In [ ]:
df_weather_test=pd.read_csv('weather_test.csv')

In [ ]:
df_weather_test_red=reduce_mem_usage(df_weather_test,verbose=True)

Mem. usage decreased to  6.08 Mb (68.1% reduction)


In [ ]:
del df_weather_test

In [ ]:
df_building=pd.read_csv('building_metadata.csv')

In [ ]:
df_building_red=reduce_mem_usage(df_building,verbose=True)

Mem. usage decreased to  0.03 Mb (60.3% reduction)


In [ ]:
df_test_build=pd.merge(df_test_red,df_building_red,how='left',on=['building_id'])

In [ ]:
df_test_merge=pd.merge(df_test_build,df_weather_test_red,how='left',on=['site_id','timestamp'])

**Checking the percentage of missing values**

In [ ]:
(df_test_merge.isnull().sum()/df_test_merge.shape[0])*100

row_id                 0.000000
building_id            0.000000
meter                  0.000000
timestamp              0.000000
site_id                0.000000
primary_use            0.000000
square_feet            0.000000
year_built            58.991597
floor_count           82.605042
air_temperature        0.532167
cloud_coverage        46.866438
dew_temperature        0.625453
precip_depth_1_hr     18.709861
sea_level_pressure     6.035901
wind_direction         7.143488
wind_speed             0.724476
dtype: float64

**Dropping the floor count as it contains more than 80% missing data**

In [ ]:
df_test_merge.drop('floor_count',axis=1,inplace=True)

In [ ]:
df_test_merge.reset_index(inplace=True)

In [ ]:
df_test_merge['timestamp']=pd.to_datetime(df_test_merge['timestamp'])

In [ ]:
df_test_merge['day']=df_test_merge['timestamp'].dt.day
df_test_merge['month']=df_test_merge['timestamp'].dt.month

In [ ]:
df_test_merge_red=reduce_mem_usage(df_test_merge,verbose=True)

Mem. usage decreased to 1988.30 Mb (26.5% reduction)


In [ ]:
del df_test_merge

In [ ]:
df_test_merge_red[['year_built','air_temperature','dew_temperature','cloud_coverage','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed']]=df_test_merge_red[['year_built','air_temperature','dew_temperature','cloud_coverage','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed']].astype(np.float32)

**FROM HERE I WILL BE IMPTUING THE MISSING VALUES**

1.   HERE THE BUILDINGS ARE PRESENT AT DIFFERENT GEOGRAPHICAL LOCATIONS DUE TO WHICH THERE ARE VARIATIONS OVER THE WEATHER CONDITIONS.

2.   SO INSTEAD OF IMPUTING IT DIRECTLY BY TAKING THE MEAN HERE I AM GROUPING IT BY THE SITE ID MONTH AND DAY WHICH PROVIDES A MORE ACCURATE DATA FOR FILLING THE MISSING VALUE.



In [ ]:
cc_fill=df_test_merge_red.groupby(['site_id','day','month'])['cloud_coverage'].median().reset_index()
cc_fill.rename(columns={'cloud_coverage':'cc_filler'},inplace=True)
cc_fill['cc_filler'].fillna(method='ffill',inplace=True)
df_test_merge_red=df_test_merge_red.merge(cc_fill,how='left',on=['site_id','day','month'])
df_test_merge_red['cloud_coverage'].fillna(df_test_merge_red['cc_filler'],inplace=True)
df_test_merge_red.drop(labels=['cc_filler'],axis=1,inplace=True)

In [ ]:
wd_fill=df_test_merge_red.groupby(['site_id','day','month'])['wind_direction'].median().reset_index()
wd_fill.rename(columns={'wind_direction':'wind_direction_filler'},inplace=True)
df_test_merge_red=df_test_merge_red.merge(wd_fill,how='left',on=['site_id','day','month'])
df_test_merge_red['wind_direction'].fillna(df_test_merge_red['wind_direction_filler'],inplace=True)
df_test_merge_red.drop(labels=['wind_direction_filler'],axis=1,inplace=True)

In [ ]:
ws_fill=df_test_merge_red.groupby(['site_id','day','month'])['wind_speed'].median().reset_index()
ws_fill.rename(columns={'wind_speed':'wind_speed_filler'},inplace=True)
df_test_merge_red=df_test_merge_red.merge(ws_fill,how='left',on=['site_id','day','month'])
df_test_merge_red['wind_speed'].fillna(df_test_merge_red['wind_speed_filler'],inplace=True)
df_test_merge_red.drop(labels=['wind_speed_filler'],axis=1,inplace=True)

In [ ]:
slp_fill=df_test_merge_red.groupby(['site_id','day','month'])['sea_level_pressure'].median().reset_index()
slp_fill.rename(columns={'sea_level_pressure':'slp_filler'},inplace=True)
slp_fill.fillna(method='ffill',inplace=True)
df_test_merge_red=df_test_merge_red.merge(slp_fill,how='left',on=['site_id','day','month'])
df_test_merge_red['sea_level_pressure'].fillna(df_test_merge_red['slp_filler'],inplace=True)
df_test_merge_red.drop(labels=['slp_filler'],axis=1,inplace=True)

In [ ]:
pd_fill=df_test_merge_red.groupby(['site_id','day','month'])['precip_depth_1_hr'].median().reset_index()
pd_fill.rename(columns={'precip_depth_1_hr':'pd_filler'},inplace=True)
pd_fill['pd_filler'].fillna(method='ffill',inplace=True)
df_test_merge_red=df_test_merge_red.merge(pd_fill,how='left',on=['site_id','day','month'])
df_test_merge_red['precip_depth_1_hr'].fillna(df_test_merge_red['pd_filler'],inplace=True)
df_test_merge_red.drop(labels=['pd_filler'],axis=1,inplace=True)

In [ ]:
yb_fill=df_test_merge_red.groupby(['site_id','day','month'])['year_built'].median().reset_index()
yb_fill.rename(columns={'year_built':'yb_filler'},inplace=True)
yb_fill['yb_filler'].fillna(method='ffill',inplace=True)
df_test_merge_red=df_test_merge_red.merge(yb_fill,how='left',on=['site_id','day','month'])
df_test_merge_red['year_built'].fillna(df_test_merge_red['yb_filler'],inplace=True)
df_test_merge_red.drop(labels=['yb_filler'],axis=1,inplace=True)

In [ ]:
df_test_merge_red['air_temperature']=df_test_merge_red['air_temperature'].interpolate(method='linear')
df_test_merge_red['dew_temperature']=df_test_merge_red['dew_temperature'].interpolate(method='linear')

**AIR-TEMPERATURE ALIGNMENT**

1.   HERE THE AIR TEMPERATURE IS NOT ALIGNED WITH THE LOCAL TIMESTAMP OF THE METER READINGS.
2.   FOR 13 DIFFERENT SITES THE AIR TEMPERATURE MAXIMIZES AROUND 19:00 PM TO 23:00 PM WHICH NEEDS TO BE CORRECTED SO THAT THE MODEL CAN BEST IDENTIFY THE ENERGY PATTERN ACCORDING TO THE TEMPERATURE VARIATIONS OVER THE HOUR.



In [ ]:
from datetime import timedelta

In [ ]:
df_test_site_0=df_test_merge_red[df_test_merge_red['site_id']==0]
df_test_site_0.reset_index(inplace=True)
df_test_site_0['timestamp_aligned']=df_test_site_0['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_test_site_0[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_test_site_0.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_test_site_0['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_test_site_0['timestamp'])].reset_index(drop=True)['air_temperature']
df_test_site_0['air_temperature_aligned']=df_test_site_0['air_temperature_aligned'].interpolate()
df_test_site_0.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_test_site_0.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_test_site_1=df_test_merge_red[df_test_merge_red['site_id']==1]
df_test_site_1.reset_index(inplace=True)
df_test_site_1.drop(['index','level_0'],axis=1,inplace=True)

In [ ]:
df_test_site_2=df_test_merge_red[df_test_merge_red['site_id']==2]
df_test_site_2.reset_index(inplace=True)
df_test_site_2['timestamp_aligned']=df_test_site_2['timestamp']-timedelta(hours=7,minutes=0)
df_air_temp_timestamp=df_test_site_2[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_test_site_2.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_test_site_2['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_test_site_2['timestamp'])].reset_index(drop=True)['air_temperature']
df_test_site_2['air_temperature_aligned']=df_test_site_2['air_temperature_aligned'].interpolate()
df_test_site_2.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_test_site_2.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_test_site_3=df_test_merge_red[df_test_merge_red['site_id']==3]
df_test_site_3.reset_index(inplace=True)
df_test_site_3['timestamp_aligned']=df_test_site_3['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_test_site_3[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_test_site_3.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_test_site_3['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_test_site_3['timestamp'])].reset_index(drop=True)['air_temperature']
df_test_site_3['air_temperature_aligned']=df_test_site_3['air_temperature_aligned'].interpolate()
df_test_site_3.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_test_site_3.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_test_site_4=df_test_merge_red[df_test_merge_red['site_id']==4]
df_test_site_4.reset_index(inplace=True)
df_test_site_4['timestamp_aligned']=df_test_site_4['timestamp']-timedelta(hours=8,minutes=0)
df_air_temp_timestamp=df_test_site_4[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_test_site_4.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_test_site_4['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_test_site_4['timestamp'])].reset_index(drop=True)['air_temperature']
df_test_site_4['air_temperature_aligned']=df_test_site_4['air_temperature_aligned'].interpolate()
df_test_site_4.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_test_site_4.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_test_site_5=df_test_merge_red[df_test_merge_red['site_id']==5]
df_test_site_5.reset_index(inplace=True)
df_test_site_5.drop(['index','level_0'],axis=1,inplace=True)

In [ ]:
df_test_site_6=df_test_merge_red[df_test_merge_red['site_id']==6]
df_test_site_6.reset_index(inplace=True)
df_test_site_6['timestamp_aligned']=df_test_site_6['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_test_site_6[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_test_site_6.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_test_site_6['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_test_site_6['timestamp'])].reset_index(drop=True)['air_temperature']
df_test_site_6['air_temperature_aligned']=df_test_site_6['air_temperature_aligned'].interpolate()
df_test_site_6.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_test_site_6.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_test_site_7=df_test_merge_red[df_test_merge_red['site_id']==7]
df_test_site_7.reset_index(inplace=True)
df_test_site_7['timestamp_aligned']=df_test_site_7['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_test_site_7[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_test_site_7.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_test_site_7['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_test_site_7['timestamp'])].reset_index(drop=True)['air_temperature']
df_test_site_7['air_temperature_aligned']=df_test_site_7['air_temperature_aligned'].interpolate()
df_test_site_7.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_test_site_7.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_test_site_8=df_test_merge_red[df_test_merge_red['site_id']==8]
df_test_site_8.reset_index(inplace=True)
df_test_site_8['timestamp_aligned']=df_test_site_8['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_test_site_8[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_test_site_8.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_test_site_8['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_test_site_8['timestamp'])].reset_index(drop=True)['air_temperature']
df_test_site_8['air_temperature_aligned']=df_test_site_8['air_temperature_aligned'].interpolate()
df_test_site_8.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_test_site_8.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_test_site_9=df_test_merge_red[df_test_merge_red['site_id']==9]
df_test_site_9.reset_index(inplace=True)
df_test_site_9['timestamp_aligned']=df_test_site_9['timestamp']-timedelta(hours=6,minutes=0)
df_air_temp_timestamp=df_test_site_9[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_test_site_9.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_test_site_9['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_test_site_9['timestamp'])].reset_index(drop=True)['air_temperature']
df_test_site_9['air_temperature_aligned']=df_test_site_9['air_temperature_aligned'].interpolate()
df_test_site_9.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_test_site_9.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_test_site_10=df_test_merge_red[df_test_merge_red['site_id']==10]
df_test_site_10.reset_index(inplace=True)
df_test_site_10['timestamp_aligned']=df_test_site_10['timestamp']-timedelta(hours=7,minutes=0)
df_air_temp_timestamp=df_test_site_10[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_test_site_10.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_test_site_10['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_test_site_10['timestamp'])].reset_index(drop=True)['air_temperature']
df_test_site_10['air_temperature_aligned']=df_test_site_10['air_temperature_aligned'].interpolate()
df_test_site_10.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_test_site_10.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_test_site_11=df_test_merge_red[df_test_merge_red['site_id']==11]
df_test_site_11.reset_index(inplace=True)
df_test_site_11['timestamp_aligned']=df_test_site_11['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_test_site_11[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_test_site_11.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_test_site_11['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_test_site_11['timestamp'])].reset_index(drop=True)['air_temperature']
df_test_site_11['air_temperature_aligned']=df_test_site_11['air_temperature_aligned'].interpolate()
df_test_site_11.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_test_site_11.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_test_site_12=df_test_merge_red[df_test_merge_red['site_id']==12]
df_test_site_12.reset_index(inplace=True)
df_test_site_12.drop(['index','level_0'],axis=1,inplace=True)

In [ ]:
df_test_site_13=df_test_merge_red[df_test_merge_red['site_id']==13]
df_test_site_13.reset_index(inplace=True)
df_test_site_13['timestamp_aligned']=df_test_site_13['timestamp']-timedelta(hours=6,minutes=0)
df_air_temp_timestamp=df_test_site_13[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_test_site_13.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_test_site_13['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_test_site_13['timestamp'])].reset_index(drop=True)['air_temperature']
df_test_site_13['air_temperature_aligned']=df_test_site_13['air_temperature_aligned'].interpolate()
df_test_site_13.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_test_site_13.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_test_site_14=df_test_merge_red[df_test_merge_red['site_id']==14]
df_test_site_14.reset_index(inplace=True)
df_test_site_14['timestamp_aligned']=df_test_site_14['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_test_site_14[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_test_site_14.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_test_site_14['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_test_site_14['timestamp'])].reset_index(drop=True)['air_temperature']
df_test_site_14['air_temperature_aligned']=df_test_site_14['air_temperature_aligned'].interpolate()
df_test_site_14.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_test_site_14.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
df_test_site_15=df_test_merge_red[df_test_merge_red['site_id']==15]
df_test_site_15.reset_index(inplace=True)
df_test_site_15['timestamp_aligned']=df_test_site_15['timestamp']-timedelta(hours=5,minutes=0)
df_air_temp_timestamp=df_test_site_15[['timestamp_aligned','building_id','meter','air_temperature']].copy()
df_air_temp_timestamp.rename(columns={'timestamp_aligned':'timestamp'},inplace=True)
df_test_site_15.drop(['air_temperature','timestamp_aligned'],axis=1,inplace=True)
df_test_site_15['air_temperature_aligned']=df_air_temp_timestamp[df_air_temp_timestamp['timestamp'].isin(df_test_site_15['timestamp'])].reset_index(drop=True)['air_temperature']
df_test_site_15['air_temperature_aligned']=df_test_site_15['air_temperature_aligned'].interpolate()
df_test_site_15.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)
df_test_site_15.drop(['level_0','index'],axis=1,inplace=True)

**END OF AIR-TEMPERATURE ALIGNMENT**

**Merging the data from all the different sites and then merging it with the original test df so that I can get the information in the correct order needed for prediction.**

In [ ]:
df_te=pd.concat([df_test_site_0,df_test_site_1,df_test_site_2,df_test_site_3,df_test_site_4,
                                 df_test_site_5,df_test_site_6,df_test_site_7,df_test_site_8,df_test_site_9,
                                 df_test_site_10,df_test_site_11,df_test_site_12,df_test_site_13,df_test_site_14,
                                 df_test_site_15],axis=0)

In [ ]:
df_te.reset_index(inplace=True)

In [ ]:
df_te.drop(['row_id','index','primary_use','square_feet','year_built','cloud_coverage','dew_temperature',
            'precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed','day','month'],axis=1,inplace=True)
df_test_merge_red.drop('index',axis=1,inplace=True)

In [ ]:
df_te.rename(columns={'air_temperature':'air_temperature_aligned'},inplace=True)

In [ ]:
df_test_merge_red_final=pd.merge(left=df_test_merge_red,right=df_te,on=['site_id','timestamp','meter','building_id'])

In [ ]:
del df_test_merge_red,df_te
del df_test_site_0,df_test_site_1,df_test_site_2,df_test_site_3,df_test_site_4,df_test_site_5,df_test_site_6,df_test_site_7,df_test_site_8,df_test_site_9,df_test_site_10,df_test_site_11,df_test_site_12,df_test_site_13,df_test_site_14,df_test_site_15

In [ ]:
df_test_merge_red_final.drop('air_temperature',axis=1,inplace=True)
df_test_merge_red_final.rename(columns={'air_temperature_aligned':'air_temperature'},inplace=True)

**FEATURE ENGINEERING**

1.   RELATIVE HUMIDITY-->HUMIDITY AFFECTS THE ENERGY REQUIREMENTS OF THE BUILDINGS.AS HUMIDITY INCREASES THE LOAD ON THE HVAC SYSTEM INCREASES DUE TO WHICH THE METER READINGS ARE AFFECTED.THIS FEATURE WILL HELP THE MODEL TO LEARN THE ENRGY REQUIREMENTS ACCORDING TO THE HUMIDITY FACTOR.

2.   IS SUMMER MONTH & IS WINTER MONTH--> CHILLED WATER HOT WATER AND STEAM READINGS SHOW CONSIDERABLE PATTERN OVER THE SEASONS.THIS FEATURE WILL HELP THE MODEL TO LEARN THAT.

3.   HOLIDAYS-->AS THE ENERGY REQUIREMENTS MIGHT BE DIFFERENT DURING THE HOLIDAY AS COMPARED TO THE OTHER DAYS INCLUDING THIS FEATURE WILL HELP THE MODEL TO LEARN THAT.

4.   BUSY-HOURS-->AS THE ENERGY REQUIREMENTS ARE DIFFERENT OVER THE DAY THAN THE NIGHT.SO TO HELP THE MODEL LEARN THAT WE CAN USE BUSY HOURS(6:00 AM TO 18:00 PM).

5.   IS WEEKDAY-->THIS FEATURE IS ADDED TO CHECK THE ENRGY REQUIREMENTS OF THE WEEKDAY AS COMPARED TO THE WEEKEND.AS OBSERVED IN THE EDA IT SHOWS A SIGNIFICANT TREND OVER THE WEEK AS COMPARED TO THE WEEKEND FOR ALL THE ENERGY METER.

6.   ADDING BASIC TIMESTAMP FEATURES SUCH AS DAY MONTH AND HOUR.



In [ ]:
saturated_vapor_pressure = 6.11 * (10**(7.5*df_test_merge_red_final['air_temperature']/(237.3+df_test_merge_red_final['air_temperature'])))
actual_vapor_pressure = 6.11 * (10**(7.5*df_test_merge_red_final['dew_temperature']/(237.3+df_test_merge_red_final['dew_temperature'])))
df_test_merge_red_final['relative_humidity']=(actual_vapor_pressure/saturated_vapor_pressure)*100

In [ ]:
df_test_merge_red_final['is_winter_month']=(df_test_merge_red_final['month'].isin([12,1,2])).astype(int)
df_test_merge_red_final['is_summer_month']=(df_test_merge_red_final['month'].isin([6,7,8])).astype(int)
df_test_merge_red_final['weekday']=df_test_merge_red_final['timestamp'].dt.weekday

In [ ]:
holidays = ["2017-01-01", "2017-01-16", "2017-02-20", "2017-05-29", "2017-07-04",
            "2017-09-04", "2017-10-09", "2017-11-10", "2017-11-23", "2017-12-25",
            "2018-01-01", "2018-01-15", "2018-02-19", "2018-05-28", "2018-07-04",
            "2018-09-03", "2018-10-08", "2018-11-12", "2018-11-22", "2018-12-25",
            "2019-01-01"]

In [ ]:
holiday_datetime=pd.to_datetime(holidays,yearfirst=True)
df_test_merge_red_final['is_pub_holiday']=(df_test_merge_red_final['timestamp'].dt.date.isin(holiday_datetime.date)).astype(int)
df_test_merge_red_final['is_weekday']=((~df_test_merge_red_final['timestamp'].dt.date.isin(holiday_datetime.date))&(df_test_merge_red_final['weekday'].isin([0,1,2,3,4]))).astype(int)

In [ ]:
z_busy_hours=df_test_merge_red_final.set_index(['timestamp']).between_time('06:00:00','18:00:00').reset_index()
z_busy_hours_timestamp=[i for i in z_busy_hours['timestamp']]
df_test_merge_red_final['busy_hours']=((~df_test_merge_red_final['timestamp'].dt.date.isin(holiday_datetime.date))&(df_test_merge_red_final['timestamp'].isin(z_busy_hours_timestamp))).astype(int)

In [ ]:
df_test_merge_red_final['hour']=df_test_merge_red_final['timestamp'].dt.hour

**End of Feature engineering**

In [ ]:
df_te_red=reduce_mem_usage(df_test_merge_red_final,verbose=True)

Mem. usage decreased to 2505.25 Mb (53.0% reduction)


In [ ]:
del df_test_merge_red_final

In [ ]:
df_te_red.drop(['timestamp'],axis=1,inplace=True)

**Label Encoding of the categorical variables**

In [ ]:
label_encoder=LabelEncoder()
df_te_red['primary_use']=label_encoder.fit_transform(df_te_red['primary_use'])

In [ ]:
df_te_red_final=reduce_mem_usage(df_te_red,verbose=True)

Mem. usage decreased to 1908.76 Mb (12.7% reduction)


In [ ]:
del df_te_red

**Storing my final test file on drive**

In [ ]:
filename='df_te_red_final_modified.txt'
my_file=open(filename,'wb')

In [ ]:
pickle.dump(df_te_red_final,my_file)

In [ ]:
#REF-->https://towardsdatascience.com/forecasting-energy-consumption-using-neural-networks-xgboost-2032b6e6f7e2
    #--->https://www.kaggle.com/gunesevitan/ashrae-eda-and-preprocessing
    #--->https://www.kaggle.com/gemartin/load-data-reduce-memory-usage